# Federated GAN tutorial with PyTorch Lightning

In [1]:
# Install dependencies if not already installed
!pip install "pytorch-lightning>=1.3" "torch==1.9.1" "torchvision==0.10.1" "torchmetrics>=0.3" "dill==0.3.4"

# Connect to the Federation

In [2]:
# Create a federation
from openfl.interface.interactive_api.federation import Federation

# please use the same identificator that was used in signed certificate
client_id = 'frontend'
director_node_fqdn = 'localhost'
director_port = 50050

# 1) Run with API layer - Director mTLS 
# If the user wants to enable mTLS their must provide CA root chain, and signed key pair to the federation interface
# cert_chain = 'cert/root_ca.crt'
# API_certificate = 'cert/frontend.crt'
# API_private_key = 'cert/frontend.key'

# federation = Federation(
#     client_id=client_id,
#     director_node_fqdn=director_node_fqdn,
#     director_port=director_port,
#     tls=True,
#     cert_chain=cert_chain,
#     api_cert=api_certificate,
#     api_private_key=api_private_key
# )

# --------------------------------------------------------------------------------------------------------------------

# 2) Run with TLS disabled (trusted environment)
# Federation can also determine local fqdn automatically
federation = Federation(
    client_id=client_id,
    director_node_fqdn=director_node_fqdn,
    director_port=director_port,
    tls=False
)


In [3]:
# import time
# while True:
#     shard_registry = federation.get_shard_registry()
#     print(shard_registry)
#     time.sleep(5)
shard_registry = federation.get_shard_registry()
shard_registry

{'env_one': {'shard_info': node_info {
    name: "env_one"
    cuda_devices {
    }
    cuda_devices {
      index: 2
    }
  }
  shard_description: "Mnist dataset, shard number 1 out of 2"
  sample_shape: "28"
  sample_shape: "28"
  target_shape: "1",
  'is_online': True,
  'is_experiment_running': False,
  'last_updated': '2021-12-29 11:00:02',
  'current_time': '2021-12-29 11:00:28',
  'valid_duration': seconds: 120,
  'experiment_name': 'ExperimentName Mock'}}

In [4]:
federation.target_shape

['1']

In [5]:
# First, request a dummy_shard_desc that holds information about the federated dataset 
# dummy_shard_desc = federation.get_dummy_shard_descriptor(size=10)
# dummy_shard_dataset = dummy_shard_desc.get_dataset('train')
# sample, target = dummy_shard_dataset[0]
# f"Sample shape: {sample.shape}, target shape: {target.shape}"

## Creating a FL experiment using Interactive API

In [6]:
from openfl.interface.interactive_api.experiment import TaskInterface, DataInterface, ModelInterface, FLExperiment

### Register dataset

In [7]:
import os
from collections import OrderedDict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST

# PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
# AVAIL_GPUS = min(1, torch.cuda.device_count())
# BATCH_SIZE = 256 if AVAIL_GPUS else 64
# NUM_WORKERS = int(os.cpu_count() / 2)

In [8]:
# class MNISTDataModule(LightningDataModule):
#     def __init__(
#         self,
#         data_dir: str = PATH_DATASETS,
#         batch_size: int = BATCH_SIZE,
#         num_workers: int = NUM_WORKERS,
#     ):
#         super().__init__()
#         self.data_dir = data_dir
#         self.batch_size = batch_size
#         self.num_workers = num_workers

#         self.transform = transforms.Compose(
#             [
#                 transforms.ToTensor(),
#                 transforms.Normalize((0.1307,), (0.3081,)),
#             ]
#         )

#         # self.dims is returned when you call dm.size()
#         # Setting default dims here because we know them.
#         # Could optionally be assigned dynamically in dm.setup()
#         self.dims = (1, 28, 28)
#         self.num_classes = 10

#     def prepare_data(self):
#         # download
#         MNIST(self.data_dir, train=True, download=True)
#         MNIST(self.data_dir, train=False, download=True)

#     def setup(self, stage=None):
#         # Assign train/val datasets for use in dataloaders
#         if stage == "fit" or stage is None:
#             mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
#             self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

#         # Assign test dataset for use in dataloader(s)
#         if stage == "test" or stage is None:
#             self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

#     def train_dataloader(self):
#         return DataLoader(
#             self.mnist_train,
#             batch_size=self.batch_size,
#             num_workers=self.num_workers,
#         )

#     def val_dataloader(self):
#         return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=self.num_workers)

#     def test_dataloader(self):
#         return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=self.num_workers)
    

In [9]:
import os
import PIL
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as tsf

# Now you can implement you data loaders using dummy_shard_desc
class MnistFedDataset(DataInterface):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @property
    def shard_descriptor(self):
        return self._shard_descriptor

    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.

        This method will be called during a collaborator initialization.
        Local shard_descriptor will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor
        
        self.train_set = shard_descriptor.get_dataset('train')
        self.valid_set = shard_descriptor.get_dataset('val')

    def __getitem__(self, index):
        return self.shard_descriptor[index]

    def __len__(self):
        return len(self.shard_descriptor)

    def get_train_loader(self):
        """
        Output of this method will be provided to tasks with optimizer in contract
        """
        if self.kwargs['train_bs']:
            batch_size = self.kwargs['train_bs']
        else:
            batch_size = 64
        return DataLoader(self.train_set, batch_size=batch_size, num_workers=36)

    def get_valid_loader(self):
        """
        Output of this method will be provided to tasks without optimizer in contract
        """
        if self.kwargs['valid_bs']:
            batch_size = self.kwargs['valid_bs']
        else:
            batch_size = 64
        
        return DataLoader(self.valid_set, batch_size=batch_size)

    def get_train_data_size(self):
        """
        Information for aggregation
        """
        
        return len(self.train_set)

    def get_valid_data_size(self):
        """
        Information for aggregation
        """
        return len(self.val_set)

In [10]:
fed_dataset = MnistFedDataset(train_bs=256, valid_bs=64)

### Describe a model and optimizer

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

In [12]:
"""
Generator and discriminator model definition
"""
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super().__init__()
        self.img_shape = img_shape

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh(),
        )

    def forward(self, z):
        z = z.float()
        img = self.model(z)
        img = img.view(img.size(0), *self.img_shape)
        return img



In [13]:
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        img_flat = img_flat.float()
        validity = self.model(img_flat)

        return validity

In [14]:
class GAN(LightningModule):
    def __init__(
        self,
        channels,
        width,
        height,
#         generator,
#         discriminator,
        latent_dim: int = 100,
        lr: float = 0.0002,
        b1: float = 0.5,
        b2: float = 0.999,
        batch_size: int = 256,
        **kwargs
    ):
        super().__init__()
        self.save_hyperparameters()

        # networks
        data_shape = (channels, width, height)
#         self.generator = generator
#         self.discriminator = discriminator
        self.generator = Generator(latent_dim=self.hparams.latent_dim, img_shape=data_shape)
        self.discriminator = Discriminator(img_shape=data_shape)

        self.validation_z = torch.randn(8, self.hparams.latent_dim)
#         self.validation_z = torch.randn(8, 100)
#         self.example_input_array = torch.zeros(2, 100)

        self.example_input_array = torch.zeros(2, self.hparams.latent_dim)

    def forward(self, z):
        return self.generator(z)

    def adversarial_loss(self, y_hat, y):
        return F.binary_cross_entropy(y_hat, y)

    def training_step(self, batch, batch_idx, optimizer_idx):
        imgs, _ = batch

        # sample noise
        z = torch.randn(imgs.shape[0], self.hparams.latent_dim)
        z = z.type_as(imgs)

        # train generator
        if optimizer_idx == 0:

            # generate images
            self.generated_imgs = self(z)

            # log sampled images
            sample_imgs = self.generated_imgs[:6]
            grid = torchvision.utils.make_grid(sample_imgs)
            self.logger.experiment.add_image("generated_images", grid, 0)

            # ground truth result (ie: all fake)
            # put on GPU because we created this tensor inside training_loop
            valid = torch.ones(imgs.size(0), 1)
#            valid = valid.type_as(imgs)
            valid = valid.type_as(imgs).float()

            # adversarial loss is binary cross-entropy
            g_loss = self.adversarial_loss(self.discriminator(self(z)), valid)
            tqdm_dict = {"g_loss": g_loss}
            output = OrderedDict({"loss": g_loss, "progress_bar": tqdm_dict, "log": tqdm_dict})
            return output

        # train discriminator
        if optimizer_idx == 1:
            # Measure discriminator's ability to classify real from generated samples

            # how well can it label as real?
            valid = torch.ones(imgs.size(0), 1)
#            valid = valid.type_as(imgs)
            valid = valid.type_as(imgs).float()

            real_loss = self.adversarial_loss(self.discriminator(imgs), valid)

            # how well can it label as fake?
            fake = torch.zeros(imgs.size(0), 1)
#             fake = fake.type_as(imgs)
            fake = fake.type_as(imgs).float()

            fake_loss = self.adversarial_loss(self.discriminator(self(z).detach()), fake)

            # discriminator loss is the average of these
            d_loss = (real_loss + fake_loss) / 2
            tqdm_dict = {"d_loss": d_loss}
            output = OrderedDict({"loss": d_loss, "progress_bar": tqdm_dict, "log": tqdm_dict})
            return output

    def configure_optimizers(self):
        print("Inside conf optimizers")
        lr = self.hparams.lr
        b1 = self.hparams.b1
        b2 = self.hparams.b2

        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=(b1, b2))
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=(b1, b2))
#         return [opt_g, opt_d], []
        return [opt_g, opt_d]

    def on_epoch_end(self):
        z = self.validation_z.type_as(self.generator.model[0].weight)

        # log sampled images
        sample_imgs = self(z)
        grid = torchvision.utils.make_grid(sample_imgs)
        self.logger.experiment.add_image("generated_images", grid, self.current_epoch)

In [15]:
data_shape = (1, 28, 28)
model = GAN(1,28,28)
# model = GAN(1,28,28,generator=Generator(latent_dim=int(100), img_shape=data_shape),
#             discriminator=Discriminator(img_shape=data_shape))

In [16]:
# This optimizer is not directly used inside the module
#optimizer = optim.Adam(model.parameters(), lr=float(0.0002), betas=(float(0.5), float(0.999)))
optimizer = model.configure_optimizers()

Inside conf optimizers


#### Register model

In [17]:
from copy import deepcopy

framework_adapter = 'openfl.plugins.frameworks_adapters.pytorch_adapter_with_multiple_opt.FrameworkAdapterPlugin'
MI = ModelInterface(model=model, optimizer=optimizer, framework_plugin=framework_adapter)

# Save the initial model state
initial_model = deepcopy(model)

### Define and register FL tasks

In [18]:
TI = TaskInterface()
import torch

import tqdm
#from openfl.component.aggregation_functions import Median

# The Interactive API supports registering functions definied in main module or imported.
def function_defined_in_notebook(some_parameter):
    print(f'Also I accept a parameter and it is {some_parameter}')

# Task interface currently supports only standalone functions.
@TI.add_kwargs(**{'some_parameter': 42})
@TI.register_fl_task(model='model', data_loader='train_loader', \
                     device='device', optimizer='optimizer')     
def train(model, train_loader, optimizer, device, some_parameter=None):
    
    print(f'\n\n TASK TRAIN GOT DEVICE {device}\n\n')
    
    function_defined_in_notebook(some_parameter)
    
    AVAIL_GPUS = 1 if device == 'cuda' else 0
    
#     train_loader = tqdm.tqdm(train_loader, desc="train")
    
#     unet_model.train()
#     unet_model.to(device)

#     losses = []
    trainer = Trainer(gpus=[3], max_epochs= 1, progress_bar_refresh_rate=20)
    trainer.fit(model = model, train_dataloaders = train_loader)

#     for data, target in train_loader:
#         data, target = torch.tensor(data).to(device), torch.tensor(
#             target).to(device, dtype=torch.float32)
#         optimizer.zero_grad()
#         output = unet_model(data)
#         loss = loss_fn(output=output, target=target)
#         loss.backward()
#         optimizer.step()
#         losses.append(loss.detach().cpu().numpy())
        
    return {'train_loss': 0.111}


# @TI.register_fl_task(model='unet_model', data_loader='val_loader', device='device')     
# def validate(unet_model, val_loader, device):
#     print(f'\n\n TASK VALIDATE GOT DEVICE {device}\n\n')
    
#     unet_model.eval()
#     unet_model.to(device)
    
#     val_loader = tqdm.tqdm(val_loader, desc="validate")

#     val_score = 0
#     total_samples = 0

#     with torch.no_grad():
#         for data, target in val_loader:
#             samples = target.shape[0]
#             total_samples += samples
#             data, target = torch.tensor(data).to(device), \
#                 torch.tensor(target).to(device, dtype=torch.int64)
#             output = unet_model(data)
#             val = soft_dice_coef(output, target)
#             val_score += val.sum().cpu().numpy()
            
#     return {'dice_coef': val_score / total_samples,}

## Time to start a federated learning experiment

In [19]:
# create an experimnet in federation
experiment_name = 'PL_MNIST_test_experiment'
serializer_plugin = 'openfl.plugins.interface_serializer.cloudpickle_serializer.CloudpickleSerializer'
#serializer_plugin = 'openfl.plugins.interface_serializer.dill_serializer.DillSerializer'
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name, serializer_plugin=serializer_plugin)

In [20]:
# If I use autoreload I got a pickling error

# The following command zips the workspace and python requirements to be transfered to collaborator nodes
fl_experiment.start(model_provider=MI, 
                    task_keeper=TI,
                    data_loader=fed_dataset,
                    rounds_to_train=5,
                    opt_treatment='CONTINUE_GLOBAL',
                    device_assignment_policy='CUDA_PREFERRED')


State is:  odict_keys(['generator.model.0.weight', 'generator.model.0.bias', 'generator.model.2.weight', 'generator.model.2.bias', 'generator.model.3.weight', 'generator.model.3.bias', 'generator.model.3.running_mean', 'generator.model.3.running_var', 'generator.model.3.num_batches_tracked', 'generator.model.5.weight', 'generator.model.5.bias', 'generator.model.6.weight', 'generator.model.6.bias', 'generator.model.6.running_mean', 'generator.model.6.running_var', 'generator.model.6.num_batches_tracked', 'generator.model.8.weight', 'generator.model.8.bias', 'generator.model.9.weight', 'generator.model.9.bias', 'generator.model.9.running_mean', 'generator.model.9.running_var', 'generator.model.9.num_batches_tracked', 'generator.model.11.weight', 'generator.model.11.bias', 'discriminator.model.0.weight', 'discriminator.model.0.bias', 'discriminator.model.2.weight', 'discriminator.model.2.bias', 'discriminator.model.4.weight', 'discriminator.model.4.bias'])
optimizer is: None


State is:  odict_keys(['generator.model.0.weight', 'generator.model.0.bias', 'generator.model.2.weight', 'generator.model.2.bias', 'generator.model.3.weight', 'generator.model.3.bias', 'generator.model.3.running_mean', 'generator.model.3.running_var', 'generator.model.3.num_batches_tracked', 'generator.model.5.weight', 'generator.model.5.bias', 'generator.model.6.weight', 'generator.model.6.bias', 'generator.model.6.running_mean', 'generator.model.6.running_var', 'generator.model.6.num_batches_tracked', 'generator.model.8.weight', 'generator.model.8.bias', 'generator.model.9.weight', 'generator.model.9.bias', 'generator.model.9.running_mean', 'generator.model.9.running_var', 'generator.model.9.num_batches_tracked', 'generator.model.11.weight', 'generator.model.11.bias', 'discriminator.model.0.weight', 'discriminator.model.0.bias', 'discriminator.model.2.weight', 'discriminator.model.2.bias', 'discriminator.model.4.weight', 'discriminator.model.4.bias'])
optimizer is: None


In [21]:
# If user want to stop IPython session, then reconnect and check how experiment is going 
# fl_experiment.restore_experiment_state(MI)

fl_experiment.stream_metrics()

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "{"created":"@1640764897.519425582","description":"Error received from peer ipv4:127.0.0.1:50050","file":"src/core/lib/surface/call.cc","file_line":1062,"grpc_message":"Socket closed","grpc_status":14}"
>